# Глубинное обучение для текстовых данных, ФКН ВШЭ

## Домашнее задание 1: Text Suggestion

__Мягкий дедлайн: 24.09 23:59__   
__Жесткий дедлайн: 27.09 23:59__

### О задании

В этом задании вам предстоит реализовать систему, предлагающую удачное продолжение слова или нескольких следующих слов в режиме реального времени по типу тех, которые используются в почте или поисковой строке. За дополнительные баллы полученную систему нужно будет обернуть в пользовательский интерфейс с помощью библиотеки [reflex](https://github.com/reflex-dev/reflex) или аналогов. В этой домашке вам не придется обучать никаких моделей, мы ограничимся n-граммной генерацией.

### Структура

Это домашнее задание состоит из двух частей: основной и бонусной. В первой вам нужно будет выполнить 5 заданий, по итогам которых вы получите минимально рабочее решение. А во второй, пользуясь тем, что вы уже сделали, реализовать полноценную систему подсказки текста с пользовательским интерфейсом. Во второй части мы никак не будем ограничивать вашу фантазию. Делайте что угодно, лишь бы в результате получился удобный фреймворк. Чем лучше у вас будет результат, тем больше баллов вы получите. Если будет совсем хорошо, то мы добавим бонусов сверху по своему усмотрению.

### Оценивание и штрафы

Максимально допустимая оценка за работу — 15 баллов. Сдавать задание после жесткого дедлайна нельзя. При сдачи решения после мягкого дедлайна за каждый день просрочки снимается по __одному__ баллу.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Весь код должен быть написан самостоятельно. Чужим кодом для пользоваться запрещается даже с указанием ссылки на источник. В разумных рамках, конечно. Взять пару очевидных строчек кода для реализации какого-то небольшого функционала можно.

Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код.

При сдаче зададания в anytask вам будет необходимо сдать весь код, а если вы возьметесь за бонусную часть, то еще отчет и видео с демонстрацией вашего UI. За основную часть можно получить до __10-ти__ баллов, а за бонусную – до __5-ти__ баллов.

In [1]:
!pip3 install nltk reflex validators jupyter ipywidgets nbmultitask

In [2]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from urllib.parse import urlparse
from typing import List, Union, Optional, Tuple
import numpy as np
import reflex as rx
from tqdm.notebook import tqdm
import validators
import re
import multiprocessing as mp
from multiprocessing import Pool
import nbmultitask
import joblib
from joblib import Parallel, delayed, cpu_count
import pickle

nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /Users/lena-
[nltk_data]     grish/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Данные

Для получения текстовых статистик используйте датасет `emails.csv`. Вы можете найти его по [ссылке](https://disk.yandex.ru/d/ikyUhWPlvfXxCg). Он содержит более 500 тысяч электронных писем на английском языке.

In [3]:
emails = pd.read_csv("emails.csv")
len(emails)

517401

In [4]:
emails

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...
...,...,...
517396,zufferli-j/sent_items/95.,Message-ID: <26807948.1075842029936.JavaMail.e...
517397,zufferli-j/sent_items/96.,Message-ID: <25835861.1075842029959.JavaMail.e...
517398,zufferli-j/sent_items/97.,Message-ID: <28979867.1075842029988.JavaMail.e...
517399,zufferli-j/sent_items/98.,Message-ID: <22052556.1075842030013.JavaMail.e...


Заметьте, что данные очень грязные. В каждом письме содержится различная мета-информация, которая будет только мешать при предсказании продолжения текста.

__Задание 1 (2 балла).__ Очистите корпус текстов по вашему усмотрению и объясните свой выбор. В идеале обработанные тексты должны содержать только текст самого письма и ничего лишнего по типу ссылок, адресатов и прочих символов, которыми мы точно не хотим продолжать текст.

> Моя логика:
> - Убираем инфу о письме (когда, кому, от кого и прочее) - контролирую это набором _плохих_ начал `trash_starts`
> - Все приводим к нижнему регистру
> - Удаляем числа (`if word.isnumeric()`) и ссылки (`if urlparse(word).scheme`)
> - Чистим пунктуацию и делим на токены: `reg.findall(text)`
> - А еще хочу, чтобы у слов был контекст, поэтому < 3 слов в строке не хочу


In [5]:
# your code here

In [6]:
stop_words = stopwords.words("english") + ['dd', 'mm', 'yy', 'yyyy']

In [7]:
def clear_text(text):
    def is_ok_word(word: str):
        if word.isnumeric() or re.search(r"\d", word):
            return False
        if "@" in word:
            return False
        if word in stop_words:
            return False
        if (
            "www" in word
            or word.startswith(("http:\\", "https:\\"))
            or word.endswith((".com", ".org", ".net"))
            or validators.url(word)
        ):
            return False
        if len(word) < 2:
            return False
        return True

    text = text.split("\n")
    clear_text = []

    trash_starts = (
        "message-id:",
        "date:",
        "from:",
        "to:",
        "subject:",
        "mime-version:",
        "content-type:",
        "content-transfer-encoding:",
        "x-from:",
        "x-to:",
        "x-cc:",
        "x-bcc:",
        "x-folder:",
        "x-origin:",
        "x-filename:",
        "full name:",
        "login ",
        "extension:",
        "office location:",
        'static ip address',
        "cc",
        "---",
        "***",
    )

    for row in text:
        row = row.strip().lower()
        if row.startswith(trash_starts) or len(row) == 0:
            continue

        row = " ".join([x for x in row.split() if is_ok_word(x)])
        reg = re.compile(r"\w+")
        row = reg.findall(row)
        row = [x for x in row if is_ok_word(x)]
        if len(row) < 3:
            continue
        clear_text.append(" ".join(row))
    return "\n".join(clear_text)

In [8]:
clear_texts = [clear_text(text) for text in tqdm(emails["message"][:100])]
clear_texts = [text for text in clear_texts if len(text.strip()) > 0]

  0%|          | 0/100 [00:00<?, ?it/s]

In [9]:
for i in range(15, 20):
    print(clear_texts[i])
    print("_" * 100)

enclosed demographics westgate site investor alliance
investor alliance says demographics similar package
san marcos received earlier
questions information requirements let know
let know interest level westgate project
property across street sagewood units san marcos
sale approved units land selling per square
foot one two remaining approved multifamily parcels west
san marcos moratorium development
several new studies looked show rents duplexes
new units going significantly higher roughly
per square foot leased entire unit lease
psf leased term individual room
property best location student housing new
project serious interest please let know
short window opportunity equity requirement
yet known would likely secure land
know question later today
president creekside builders llc
____________________________________________________________________________________________________
meeting tuesday oct regarding
storage strategies west please mark calendars
ena denver office
_______________

Для следующего задания вам нужно будет токенизировать текст. Для этого просто разбейте его по словам. Очевидно, итоговый результат для финального пользователя будет лучше, если ваша система также будет предлагать уместную пунктуацию. Но если вы заметите, что из-за этого падает качество самого текса, то можете удалить все небуквенные символы на этапе токенизации.

In [10]:
clear_texts = [text.split() for text in clear_texts]

## Общая схема решения

Мы хотим сделать систему, которая будет ускорять набор текста, советуя подходящие продолжения. Для подсказки следующего слова мы будем использовать n-граммную модель. Так как n-граммная модель работает с целыми словами, а советы мы хотим давать в риал-тайме даже когда слово еще не дописано, сперва надо научиться дополнять слово до целого.

## Дополнение слова

В этой части вам предстоит реализовать метод дополнения слова до целого по его началу (префиксу). Для этого сперва необходимо научиться находить все слова, имеющие определенный префикс. Мы будем вызывать функцию поиска подходящих слов после каждой напечатанной пользователем буквы. Поэтому нам очень важно, чтобы поиск работал как можно быстрее. Простой перебор всех слов занимает $O(|V| \cdot n)$ времени, где $|V|$ – размер словаря, а $n$ – длина префикса. Мы же напишем [префиксное дерево](https://ru.wikipedia.org/wiki/Префиксное_дерево), которое позволяет искать слова не больше чем за $O(n + mk)$, где $m$ - число подходящих слов, а $k$ – длина суффикса.

__Задание 2 (2 балла).__ Допишите префиксное дерево для поиска слов по префиксу.

In [11]:
class PrefixTreeNode:
    def __init__(self, word: str = ""):
        self.children: dict[str, PrefixTreeNode] = {}
        self.word = word
        self.is_end_of_word = False

    def get_all_children(self):
        all_children = []
        if self.is_end_of_word:
            all_children.append(self)
        for char in self.children:
            child = self.children[char]
            if child.is_end_of_word:
                all_children.append(child)
            all_children.extend(child.get_all_children())
        return all_children


class PrefixTree:
    def __init__(self, vocabulary: List[str]):
        """
        vocabulary: список всех уникальных токенов в корпусе
        """
        self.root = PrefixTreeNode()

        # your code here
        for word in vocabulary:
            node = self.root
            for char in word:
                if char not in node.children:
                    node.children[char] = PrefixTreeNode(node.word + char)
                node = node.children[char]
            node.is_end_of_word = True

    def search_prefix(self, prefix) -> List[str]:
        """
        Возвращает все слова, начинающиеся на prefix
        prefix: str – префикс слова
        """

        # your code here
        node = self.root
        for char in prefix:
            if char not in node.children:
                return []
            node = node.children[char]
        children = node.get_all_children()
        children_words = [child.word for child in children]

        return list(set(children_words))

In [12]:
vocabulary = ["aa", "aaa", "abb", "bba", "bbb", "bcd"]
prefix_tree = PrefixTree(vocabulary)

assert set(prefix_tree.search_prefix("a")) == set(["aa", "aaa", "abb"])
assert set(prefix_tree.search_prefix("bb")) == set(["bba", "bbb"])

Теперь, когда у нас есть способ быстро находить все слова с определенным префиксом, нам нужно их упорядочить по вероятности, чтобы выбирать лучшее. Будем оценивать вероятность слова по частоте его __встречаемости в корпусе__.

__Задание 3 (2 балла).__ Допишите класс `WordCompletor`, который формирует словарь и префиксное дерево, а так же умеет находить все возможные продолжения слова вместе с их вероятностями. В этом классе вы можете при необходимости дополнительно отфильтровать слова, например, удалив все самые редкие. Постарайтесь максимально оптимизировать ваш код.

In [13]:
class WordCompletor:
    def __init__(self, corpus, border=0.0):
        self.word_counts = {}
        self.n_corpus_words = 0
        for text in tqdm(corpus):
            if isinstance(text, str):
                text = text.split()
            self.n_corpus_words += len(text)
            for word in text:
                if word not in self.word_counts:
                    self.word_counts[word] = 0
                self.word_counts[word] += 1
        vocab = list(self.word_counts.keys())
        if border < 1.0:
            abs_border = np.quantile(list(self.word_counts.values()), border)
        else:
            abs_border = round(border)
        for word in vocab:
            if self.word_counts[word] < abs_border:
                self.word_counts.pop(word)
        vocab = list(self.word_counts.keys())
        self.prefix_tree = PrefixTree(vocab)
        print('vocab size:', len(vocab))
        print('WordCompletor.__init__ finished')

    def get_words_and_probs(
        self, prefix: str, max_words: Union[int, None] = None
    ) -> Tuple[List[str], List[float]]:
        words = self.prefix_tree.search_prefix(prefix)
        probs = [self.word_counts[word] / self.n_corpus_words for word in words]

        probs_argsort = np.asarray(probs).argsort()[::-1]
        if max_words is not None:
            probs_argsort = probs_argsort[:max_words]

        words = (np.asarray(words, dtype="str")[probs_argsort]).tolist()
        probs = (np.asarray(probs)[probs_argsort]).tolist()

        return words, probs

In [14]:
dummy_corpus = [
    ["aa", "ab"],
    ["aaa", "abab"],
    ["abb", "aa", "ab", "bba", "bbb", "bcd"],
]

word_completor = WordCompletor(dummy_corpus)
words, probs = word_completor.get_words_and_probs("a")
words_probs = list(zip(words, probs))
assert set(words_probs) == {
    ("aa", 0.2),
    ("ab", 0.2),
    ("aaa", 0.1),
    ("abab", 0.1),
    ("abb", 0.1),
}

  0%|          | 0/3 [00:00<?, ?it/s]

vocab size: 8
WordCompletor.__init__ finished


## Предсказание следующих слов

Теперь, когда мы умеем дописывать слово за пользователем, мы можем пойти дальше и предожить ему следующее слово (или несколько) с учетом дописанного. Для этого мы воспользуемся n-граммной моделью.

Напомним, что вероятность последовательности для такой модели записывается по формуле
$$
P(w_1, \dots, w_T) = \prod_{i=1}^T P(w_i \mid w_{i-1}, \dots, w_{i-n}).
$$

$P(w_i \mid w_{i-1}, \dots, w_{i-n})$ оценивается по частоте встречаемости n-граммы.   

__Задание 4 (2 балла).__ Напишите класс для n-граммной модели. Никакого сглаживания добавлять не надо, мы же не хотим, чтобы модель советовала случайные слова (хоть и очень редко).

In [15]:
class NGramLanguageModel:
    def __init__(self, corpus, n):
        # your code here
        self.n = n
        self.ngram_counts = {}
        self.context_counts = {}

        for text in tqdm(corpus):
            if isinstance(text, str):
                text = text.split()
            for i in range(len(text) - n):
                context = tuple(text[i: i + n])
                next_word = text[i + n]
                context_extended = context + (next_word,)

                if context_extended not in self.ngram_counts:
                    self.ngram_counts[context_extended] = 0
                self.ngram_counts[context_extended] += 1

                if context not in self.context_counts:
                    self.context_counts[context] = 0
                self.context_counts[context] += 1

    def get_next_words_and_probs(self, prefix: list, max_words: Union[int, None] = None) -> Tuple[List[str], List[float]]:
        """
        Возвращает список слов, которые могут идти после prefix,
        а так же список вероятностей этих слов.
        Возвращает не более max_words слов.
        """

        next_words, probs = [], []

        # your code here

        if len(prefix) < self.n:
            return next_words, probs

        prefix = tuple(prefix[-self.n:])

        if prefix not in self.context_counts:
            return next_words, probs

        for context in self.ngram_counts:
            if context[: self.n] == prefix:
                next_word = context[self.n]
                next_words.append(next_word)
                probs.append(self.ngram_counts[context] / self.context_counts[prefix])

        probs_argsort = np.asarray(probs).argsort()[::-1]
        if max_words is not None:
            probs_argsort = probs_argsort[:max_words]

        next_words = (np.asarray(next_words, dtype='str')[probs_argsort]).tolist()
        probs = (np.asarray(probs)[probs_argsort]).tolist()

        return next_words, probs

    def get_next_seqs_and_probs(self,
                                prefix: list,
                                seq_len: int = 1,
                                max_seqs: Union[int, None] = None,
                                max_best_words: Union[int, None] = None) -> Tuple[List[List[str]], List[float]]:
        """
        Возвращает список продолжений для prefix, продолжение состоит из seq_len слов.
        Возвращает не более max_seqs вариантов продолжений.
        """
        next_seqs, probs = [], []
        next_words, word_probs = self.get_next_words_and_probs(prefix, max_best_words)

        if seq_len == 1:
            return [[word] for word in next_words], word_probs

        for (word, word_prob) in zip(next_words, word_probs):
            local_seqs, local_probs = self.get_next_seqs_and_probs(prefix + [word], seq_len - 1, max_seqs,
                                                                   max_best_words)
            local_seqs = [[word] + local_seq for local_seq in local_seqs]
            local_probs = [word_prob * local_prob for local_prob in local_probs]

            next_seqs.extend(local_seqs)
            probs.extend(local_probs)

        probs_argsort = np.asarray(probs).argsort()[::-1]
        if max_seqs is not None:
            probs_argsort = probs_argsort[:max_seqs]

        next_seqs = (np.asarray(next_seqs, dtype='str')[probs_argsort]).tolist()
        probs = (np.asarray(probs)[probs_argsort]).tolist()

        return next_seqs, probs

In [16]:
dummy_corpus = [
    ["aa", "aa", "aa", "aa", "ab"],
    ["aaa", "abab"],
    ["abb", "aa", "ab", "bba", "bbb", "bcd"],
]

n_gram_model = NGramLanguageModel(corpus=dummy_corpus, n=2)

next_words, probs = n_gram_model.get_next_words_and_probs(["aa", "aa"])
words_probs = list(zip(next_words, probs))

assert set(words_probs) == {("aa", 2 / 3), ("ab", 1 / 3)}

  0%|          | 0/3 [00:00<?, ?it/s]

Отлично, мы теперь можем объединить два метода в автоматический дописыватель текстов: первый будет дополнять слово, а второй – предлагать продолжения. Хочется, чтобы предлагался список возможных продолжений, из который пользователь сможет выбрать наиболее подходящее. Самое сложное тут – аккуратно выбирать, что показывать, а что нет.   

__Задание 5 (2 балла).__ В качестве первого подхода к снаряду реализуйте метод, возвращающий всегда самое вероятное продолжение жадным способом. После этого можно добавить опцию генерации нескольких вариантов продолжений, что сделает метод гораздо лучше.

In [17]:
class TextSuggestion:
    def __init__(self, word_completor: WordCompletor, n_gram_model: NGramLanguageModel):
        self.word_completor = word_completor
        self.n_gram_model = n_gram_model

    def suggest_text(
        self,
        text: Union[str, list],
        n_words: int = 3,
        n_texts: int = 1,
        max_complete_words: Union[int, None] = 1,
        n_best_words: int = 1,
        return_probs: bool = False,
    ) -> (List[List[str]], Optional[List[float]]): # type: ignore
        """
        Возвращает возможные варианты продолжения текста (по умолчанию только один)

        text: строка или список слов – написанный пользователем текст
        n_words: число слов, которые дописывает n-граммная модель
        n_texts: число возвращаемых продолжений
        max_complete_words: число вариантов дополнения последнего слова
        n_best_words: число вариантов, которые n-граммная модель предлагает для каждого слова

        return: list[list[srt]] – список из n_texts списков слов, по 1 + n_words слов в каждом
        Первое слово – это то, которое WordCompletor дополнил до целого.
        """

        # your code here

        if isinstance(text, str):
            text = text.split()

        # дописываем последнее слово
        complete_words, complete_probs = self.word_completor.get_words_and_probs(
            text[-1], max_complete_words
        )

        if not complete_words:
            return [[text[-1]] for _ in range(n_texts)]

        # для каждого варианта дополнения слова напишем продолжение
        n_sugg_per_completed = int(max(3, np.ceil(2 * n_texts / len(complete_words))))
        suggestions = []
        suggestions_probs = []

        for ind_completed, completed_word in enumerate(complete_words):
            history = text[:-1] + [completed_word]
            local_sugg, local_sugg_probs = self.n_gram_model.get_next_seqs_and_probs(
                history, n_words, n_sugg_per_completed, n_best_words
            )
            local_sugg = [[completed_word] + seq for seq in local_sugg]
            local_sugg_probs = [local_prob *  complete_probs[ind_completed] for local_prob in local_sugg_probs]

            suggestions.extend(local_sugg)
            suggestions_probs.extend(local_sugg_probs)

        probs_argsort = np.asarray(suggestions_probs).argsort()[::-1]
        probs_argsort = probs_argsort[:n_texts]

        suggestions = (np.asarray(suggestions, dtype="str")[probs_argsort]).tolist()
        suggestions_probs = (np.asarray(suggestions_probs)[probs_argsort]).tolist()

        if return_probs:
            return suggestions, suggestions_probs
        return suggestions

In [18]:
dummy_corpus = [
    ["aa", "aa", "aa", "aa", "ab"],
    ["aaa", "abab"],
    ["abb", "aa", "ab", "bba", "bbb", "bcd"],
]

word_completor = WordCompletor(dummy_corpus)
n_gram_model = NGramLanguageModel(corpus=dummy_corpus, n=2)
text_suggestion = TextSuggestion(word_completor, n_gram_model)

assert text_suggestion.suggest_text(["aa", "aa"], n_words=3, n_texts=1) == [
    ["aa", "aa", "aa", "aa"]
]
assert text_suggestion.suggest_text(["abb", "aa", "ab"], n_words=2, n_texts=1) == [
    ["ab", "bba", "bbb"]
]

  0%|          | 0/3 [00:00<?, ?it/s]

vocab size: 8
WordCompletor.__init__ finished


  0%|          | 0/3 [00:00<?, ?it/s]

In [19]:
text_suggestion = TextSuggestion(word_completor, n_gram_model)

## Бонусная часть: Добавляем UI

Запускать ячейки в юпитере – это хорошо, но будет лучше, если вашим решением действительно можно будет пользоваться. Для этого вам предлагается добавить полноценных User Interface. Мы рекомендуем использовать для этого [reflex](https://github.com/reflex-dev/reflex). Это Python библиотека для создания web-интерфейсом с очень богатым функционалом.

Ваша задача – сделать поле для текстового ввода, при наборе текста в котором будут появляться подсказки в реальном времени. Продумайте, как пользователь будет выбирать подсказки, сколько продолжений рекомендавать и так далее. В общем, должно получиться красиво и удобно. В этой части вы можете модифицировать все классы по своему усмотрению и добавлять любые эвристики. Если нужно, то дополнительно обрабатывать текст и вообще делать все, что считаете нужным. 

За это задание можно получить до __5-ти бонусных баллов__ в зависимости о того, насколько хорошо и удобно у вас получилось. При сдаче задания прикрепите небольшой __отчет__ (полстраницы) с описанием вашей системы, а также __видео__ (1-2 минуты) с демонстрацией работы интерфейса.

Мы настоятельно рекомендуем вам оформить код в проект, а не писать в ноутбуке. Но если вам очень хочется писать тут, то хотя бы не меняйте код в предыдущих заданиях, чтобы его можно было нормально оценивать.

> Сначала проверим работоспособность метода, без интерфейса

In [20]:
N_CORES = cpu_count()

list_array = np.array_split(emails["message"], N_CORES)
clear_texts = Parallel(n_jobs=N_CORES, verbose=10)(
    delayed(lambda array: list(map(clear_text, array)))(array)
    for array in np.array_split(emails["message"], N_CORES)
)
clear_texts = [x for x in sum(clear_texts, []) if len(x) > 0]

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed: 19.7min remaining: 59.2min
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed: 21.0min remaining: 35.0min
[Parallel(n_jobs=8)]: Done   4 out of   8 | elapsed: 21.0min remaining: 21.0min
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed: 21.3min remaining: 12.8min
[Parallel(n_jobs=8)]: Done   6 out of   8 | elapsed: 23.8min remaining:  7.9min
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed: 24.9min finished


In [28]:
corpus_border = 5
n = 3

In [29]:
word_completor = WordCompletor(clear_texts, corpus_border)

# save
with open(f"word_completor.pickle", "wb") as file:
    pickle.dump(word_completor, file)

  0%|          | 0/500647 [00:00<?, ?it/s]

vocab size: 113296
WordCompletor.__init__ finished


In [30]:
n_gram_model = NGramLanguageModel(corpus=clear_texts, n=n)

# save
with open(f"n_gram_model.pickle", "wb") as file:
    pickle.dump(n_gram_model, file)

  0%|          | 0/500647 [00:00<?, ?it/s]

In [31]:
import pickle
# load them
with open(f"word_completor.pickle", "rb") as file:
    word_completor = pickle.load(file)
with open(f"n_gram_model.pickle", "rb") as file:
    n_gram_model = pickle.load(file)

In [32]:
text_suggestion = TextSuggestion(word_completor, n_gram_model)
# save
with open(f"text_suggestion.pickle", "wb") as file:
    pickle.dump(text_suggestion, file)

In [33]:
text_suggestion.suggest_text('retention filtration pon'.split(), n_texts=5, max_complete_words=3, n_best_words=3, return_probs=True)

([['pond', 'requirement', 'used', 'data'],
  ['pond', 'street', 'widening', 'even']],
 [2.2107073808934363e-06, 2.2107073808934363e-06])

> отдельный GitHub для бонуса: https://github.com/GrishHelen/DL_NLP_TextSugg.git
> 
> Отчет я положила в README.md, демонстрация лежит отдельным файлом